# <center>DATA Cleaning</center>

## <center><u>``Resumo do código``</u></center>

>### Entra: 1Ninjas e 2Delta  
>### Sai: 1Ninjas_Limpo e 2Delta_Limpo

---
O objectivo é receber dados dos Ninjas e da Delta e devolver ficheiros prontos para ser trabalhados. 

Devolve os ficheiros que entram nos códigos 1 e 2.

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> **Dados dos Ninjas** limpos

> **Dados da Delta** limpos

In [1]:
import pandas as pd
import numpy as np
import csv

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)
    

In [2]:
1-9/77

0.8831168831168831

# Ler e Mudar nomes

- # `Ninjas`

In [100]:
#Info Ninja
dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas.xlsx")

#Renomear colunas
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

#Passar para datetime
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'])

- Lista de lojas

In [101]:
ficheiroLojas = pd.DataFrame(dfNinjas["STORE"].unique().tolist())
#ficheiroProdutos = pd.DataFrame(dfNinjas["DESC_ARTIGO"].unique().tolist())

- #  `Delta (stocks, reposição e sellouts)`

In [103]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta1=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta1.xlsx")
dfDelta2=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta2.xlsx")
dfDelta3=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta3.xlsx")
dfDelta4=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta4.xlsx")

dataframes=[dfDelta1,dfDelta2,dfDelta3,dfDelta4]
dfDelta=pd.concat(dataframes, ignore_index=True)

# Renomear
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%d-%m-%Y')

CPU times: total: 25.4 s
Wall time: 35.3 s


> - **Definir os produtos e lojas em causa**

In [104]:
# Ler ficheiros com os nomes de produtos e números de loja
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\lojas.csv', header=None) 

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

# Estabelecer o Dataframe mais limitado
dfDelta_Limpo = dfDelta[dfDelta['STORE'].isin(lojas) & dfDelta['DESC_ARTIGO'].isin(produtos)].copy()

> - **Limitar os dados apresentados e Renomear colunas**

In [107]:
# Tirar apenas as colunas de interesse 
dfDelta_Limpo = dfDelta.loc[:, ['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]].copy()

# Mudar nomes
dfDelta_Limpo = dfDelta_Limpo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", "SOH": "STOCK_1_Dias_Antes"})

> - **Criar colunas de Stock e Sellout para o próprio dia**

In [108]:
# Organizar para atingir os valores relevantes
dfDelta_Limpo = dfDelta_Limpo.sort_values(by=["STORE", "DESC_ARTIGO", "DATA"])

#Agrupar por loja e produto para não haver valores errados 
dfDelta_Limpo["SELLOUT"] = dfDelta_Limpo.groupby(["DESC_ARTIGO","STORE"])["SELLOUT_1_Dias_Antes"].shift(-1)
dfDelta_Limpo["STOCK"] = dfDelta_Limpo.groupby(["DESC_ARTIGO","STORE"])["STOCK_1_Dias_Antes"].shift(-1)

> - **Retirar todos os dias que não têm interesse**

In [110]:
# O dia 29 foi utilizado para conseguir dados do dia para stocks e sellouts mas não é dia de missão
dfDelta_Limpo = dfDelta_Limpo.loc[dfDelta_Limpo['DATA'] <'2023-05-29'].copy()

In [47]:
df_Fundido=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\Fundido.xlsx")

In [46]:
# Criar coluna com o número dos produtos em causa presentes no instante

df_Fundido["Num_Produtos"] = df_Fundido[produtos].sum(axis=1)

df_Fundido.loc[(df_Fundido["Num_Produtos"] == 0) & (df_Fundido[produtos[1]].isna()), "Num_Produtos"] = np.nan

In [39]:
df_Mergido= pd.merge(dfDelta1,dfFalta, how="left", on=["STORE"])

In [49]:
escrever_excel(df_Fundido, "Fundido")

In [27]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()

In [48]:
#df_Fundido["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente c/ Stock",
           2: "Ausente c/ Stock",
           3: "Presente s/ Stock",
           4: "Ausente s/ Stock"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    df_Fundido[sinal] = np.where((df_Fundido[stock] > 0) & (df_Fundido[i] == 1), 1,
                     np.where((df_Fundido[stock] > 0) & (df_Fundido[i] == 0), 2,
                     np.where((df_Fundido[stock] <= 0) & (df_Fundido[i] == 1), 3,
                     np.where((df_Fundido[stock] <= 0) & (df_Fundido[i] == 0), 4, 
                     np.nan))))


    # Map the numbers to the corresponding strings
    df_Fundido[sinal] = df_Fundido[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

In [33]:
df_columns = dfDelta1.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]
indexed_columns

['0: Vendedor',
 '1: Hora',
 '2: DATA',
 '3: Loja',
 '4: STORE',
 '5: CAFÉ DELTA Q QALIDUS 10CAP',
 '6: CAFÉ DELTA Q MYTHIQ 10CAP',
 '7: CAFÉ DELTA Q QHARACTER 10CAP',
 '8: CAFÉ GINGA 10CAP',
 '9: CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G',
 '10: CAFÉ DELTA RITUAL MU 220G',
 '11: CAFÉ TORRADO RITUAL DELTA GRÃO  500G',
 '12: CAFÉ DELTA MOAGEM UNIVERSAL BRASIL 220G',
 '13: CAFÉ DELTA MOAGEM UNIVERSAL ANGOLA 220G',
 '14: CAFÉ SOLÚVEL DELTA FRASCO 200G',
 '15: BEBIDA CEREAIS DELTA C/20%CAFE FR 200G',
 '16: Num_Produtos',
 '17: Reposição',
 '18: STOCK CAFÉ DELTA Q QALIDUS 10CAP',
 '19: PRESLINEAR CAFÉ DELTA Q QALIDUS 10CAP',
 '20: INTRANSIT CAFÉ DELTA Q QALIDUS 10CAP',
 '21: EXPECTED CAFÉ DELTA Q QALIDUS 10CAP',
 '22: SELLOUT CAFÉ DELTA Q QALIDUS 10CAP',
 '23: STOCK CAFÉ DELTA Q MYTHIQ 10CAP',
 '24: PRESLINEAR CAFÉ DELTA Q MYTHIQ 10CAP',
 '25: INTRANSIT CAFÉ DELTA Q MYTHIQ 10CAP',
 '26: EXPECTED CAFÉ DELTA Q MYTHIQ 10CAP',
 '27: SELLOUT CAFÉ DELTA Q MYTHIQ 10CAP',
 '28: STOCK CAFÉ DELTA Q QHARA

In [11]:
column_to_move = dfDelta1.iloc[:, 17]  # Extract the column at index 17 (Python is 0-indexed)

# Remove the column from its current position
dfDelta1.drop(dfDelta1.columns[17], axis=1, inplace=True)

# Shift the remaining columns to the right by one position
dfDelta1 = pd.concat([column_to_move, dfDelta1], axis=1)

# Escrever

In [111]:
escrever_excel(dfNinjas, "1Ninjas_Limpo")
escrever_excel(dfDelta_Limpo, "2Delta_Limpo")

#escrever_csv(ficheiroLojas, "lojas")
#escrever_csv(ficheiroProdutos, "produtos")

# Resumo do código

### <u>Código que gera o ficheiro OUT_1_DataFusion</u>
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um conjunto de métricas para estudar o comportamento de vários produtos em diversas lojas. 

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Long
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Long__ (artigos na mesma coluna)

> __Métricas novas:__
> - Sinal
> - Ciclos e Adequação de Stock
> - Restock


In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dataFrame, nomeFicheiro):
    dataFrame.to_excel('%s.xlsx' %nomeFicheiro, index=False)

- # `Ninjas`

In [11]:
#Info Ninja

dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas_Limpo.xlsx")

CPU times: total: 31.2 ms
Wall time: 45.1 ms


> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [12]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

In [7]:
# Nomes de entrada
x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
y= input("Último produto: ")                  # Delta Cafés Mistura + café 200grs

# Passar para uma lista
produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
resto = dfNinjas.columns.difference(produtos)

Primeiro produto: CAFÉ DELTA Q MYTHIQ 80CAP
Último produto: Delta Cafés Mistura + café 200grs


> Reposição

In [26]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna de reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfNinjas['STORE']]

> Vendedor

In [23]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\B&N Dados\Delta\Padrão_05_2023\\Vendedor.xlsx", sheet_name = "Lista Lojas Sonae")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

- # `Delta`

In [4]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta.xlsx", sheet_name="Sheet1")

# Renomear
dfDelta = dfDelta.drop(columns="STORE_NAME")

## O que tem cada ficheiro:

### **dfDelta**:
1. Formato long (produtos numa única coluna "DESC_ARTIGO")
2. Stocks, em Trânsito, Esperado, Linear, sellout dia anterior	


### **dfNinjas**:
1. Formato wide (cada produto numa coluna)
2. Presença de produto em linear para cada produto

### **dfSonae**:
1. Formato wide
2. Sellouts, Vendedores, Reposição


#  <span style="color:green"><u> Mergir</u> </span>

In [9]:
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
dfMeio = dfMeio.drop(columns=["Loja"])

In [ ]:
dfFinal=dfMeio.copy()

###  <span style="color:green">Fim</span>

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> STK

In [ ]:
# Definir coluna de STK (soma de stocks com stock em trânsito)

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

> Balanço

In [ ]:
# Definir coluna de Balanço (razão entre sellout e stock total)

dfFinal["Balanço"] = dfFinal["SELLOUT"] / dfFinal["STK"]

> Rotura

In [15]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

>Sinal

In [16]:
dfFinal["Sinal"]= pd.Series(dtype='int')

dfFinal['Sinal'] = np.where(dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), "Verdadeiramente Correcto",
                   np.where(~dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), "Falso Positivo",
                   np.where(dfFinal['NinjaInfo'].astype(bool) & ~dfFinal['SOH'].astype(bool), "Falso Negativo", 
                   "Rejeição Correcta")))

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

> Ciclos

In [17]:
dfFinal["Ciclos"]=dfFinal["SOH"]/dfFinal["PRES_STOCK"]

> Adequação

In [18]:
dfFinal["Adequação"] = np.where(
                        dfFinal["Ciclos"] > 1.1,
                        "Stock Suficiente",
                       np.where(
                        (dfFinal["Ciclos"] <= 1.1) & (dfFinal["INTRANSIT"] + dfFinal["EXPECTED"] + dfFinal["SOH"] >= dfFinal["PRES_STOCK"]),
                        "Stock Insuf c Forn Adequado",
                       np.where(
                        (dfFinal["Ciclos"] <= 1.1) & (dfFinal["INTRANSIT"] + dfFinal["EXPECTED"] + dfFinal["SOH"] < dfFinal["PRES_STOCK"]),
                        "Stock Insuf c Forn Desadequado",
                        ""
        )
    )
)


> Dias para a rotura de stock

In [ ]:
dfFinal["Dias_para_Rotura"] = dfFinal["Stock"] / dfFinal["SELLOUT"]

> Dias para a rotura de prateleira

In [ ]:
dfFinal["Dias_para_Rotura_Linear"] = dfFinal["PRES_STOCK"] / dfFinal["SELLOUT"]

- Roturas Linear (Ninjas)

In [28]:
#1 Quando detectada rotura
dfFinal["Roturas_reais_dia"] = dfFinal[dfFinal['NinjaInfo'] == 0].groupby(['STORE', 'DATA', 'DESC_ARTIGO'])['NinjaInfo'].transform('count')
#0 Quando não há rotura
dfFinal["Roturas_reais_dia"] = np.where(dfFinal['NinjaInfo'] == 1, 0, dfFinal['Roturas_reais_dia'])

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Sinal 
- Ciclos
- Adequação de Stock


# Reomear e organizar

# Escrever

In [36]:
df_columns = dfFinalCorr.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]
indexed_columns

['0: DATA',
 '1: Hora',
 '2: Altura_do_Dia',
 '3: Dia',
 '4: Semana',
 '5: DESC_ARTIGO',
 '6: EAN',
 '7: Nome da Loja',
 '8: STORE',
 '9: NinjaInfo',
 '10: SOH',
 '11: INTRANSIT',
 '12: EXPECTED',
 '13: PRES_STOCK',
 '14: Dia_Actual',
 '15: 1_Dia_Antes',
 '16: Reposição',
 '17: Rotura',
 '18: Sinal',
 '19: Ciclos',
 '20: Adequação',
 '21: Restock',
 '22: Roturas_reais_dia',
 '23: Roturas_Consecutivas_dia',
 '24: Roturas_Consecutivas_fds',
 '25: Proporção_Consecutivas_dia',
 '26: Std_Consecutivas_dia',
 '27: Proporção_Consecutivas_fds',
 '28: Std_Consecutivas_fds']

In [33]:
dfFinal=dfFinal.rename(columns={"VND (D-1)": "1_Dia_Antes", "Quant": "Sellout_actual"})
dfFinalCorr=dfFinal.iloc[:, [0,4,3,15,14,2,7,5,1,6,8,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28]]

In [84]:
escrever_excel(dfFinalCorr, "OUT_1_DeltaFusion")